In [2]:
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
import torch
from tqdm.auto import tqdm
torch.cuda.is_available()

True

# Step 1: Prepare documents

In [16]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [17]:
#docs_raw

In [18]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

# Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [ ]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"


# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch (cuda version)
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2", device='cuda')

In [ ]:
len(model.encode("This is a simple sentence"))

In [ ]:
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

# Step 3: Setup ElasticSearch connection

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'd8f7ced0e27c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qyzBw4GHSaaVZAlwLeZukA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Step 4: Create Mappings and Index

- Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

- Each document is a collection of fields, which each have their own data type.

- We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [9]:
index_name = "course-questions"

In [ ]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

# Step 5: Add documents into index

In [ ]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 6: Create end user query

In [24]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [ ]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}
res = es_client.search(index=index_name, knn=knn_query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

In [ ]:
query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": search_term,
                "fields": ["text", "section", "question", "course"],
                "type": "best_fields"
            }
        },
        "filter": {
            "term": {
                "course": "data-engineering-zoomcamp"
            }
        }
    }
}
res = es_client.search(index=index_name, query=query)
res["hits"]["hits"]

# Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [21]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [25]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [ ]:
response["hits"]["hits"]